In [1]:
# !pip install nltk
# !pip install pandas
# !pip install sklearn
# !pip install gensim
# !pip install h5py

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import sklearn

# clean up warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Allen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Allen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Allen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
import dak
traindf, testdf, word2vec_traindf = dak.init_review()

Training set:
   sentiment                                             review
0          1  With all this stuff going down at the moment w...
1          1  \The Classic War of the Worlds\" by Timothy Hi...
2          0  The film starts with a manager (Nicholas Bell)...
3          0  It must be assumed that those who praised this...
4          1  Superbly trashy and wondrously unpretentious 8...
Test set:
                                              review
0  Naturally in a film who's main themes are of m...
1  This movie is a disaster within a disaster fil...
2  All in all, this is a movie for kids. We saw i...
3  Afraid of the Dark left me with the impression...
4  A very accurate depiction of small time mob li...
Unlabeled set:
                                              review
0  Watching Time Chasers, it obvious that it was ...
1  I saw this film about 20 years ago and remembe...
2  Minor Spoilers<br /><br />In New York, Joan Ba...
3  I went to see this film with a great deal of

In [3]:
traindf = dak.preprocess_reviews(traindf)
testdf = dak.preprocess_reviews(testdf)
word2vec_traindf = dak.preprocess_reviews(word2vec_traindf)

Training number of reviews:  25000
Training number of reviews:  25000
Training number of reviews:  99999


In [5]:
wordvec_model = dak.init_word2vec(word2vec_traindf)

In [6]:
print("Vocabulary size:", len(wordvec_model.wv.vocab))
print("Number of reviews: ",len(word2vec_traindf))
wordvec_model.wv.most_similar("great")

Vocabulary size: 76819
Number of reviews:  99999


[('excellent', 0.696182131767273),
 ('fantastic', 0.6887458562850952),
 ('wonderful', 0.6717764139175415),
 ('terrific', 0.6651856899261475),
 ('good', 0.6206774115562439),
 ('awesome', 0.5967397689819336),
 ('fine', 0.5825192928314209),
 ('outstanding', 0.5730646848678589),
 ('superb', 0.5645232200622559),
 ('amaze', 0.561234712600708)]

In [13]:
from keras.preprocessing.sequence import pad_sequences

embed_size = 150
X_pad = pad_sequences(
    sequences=dak.vectorize_data(traindf['processed_review'], vocab=wordvec_model.wv.vocab),
    maxlen=embed_size,
    padding='post')

AttributeError: module 'dak' has no attribute 'vectorize_data'

In [9]:
y = traindf['sentiment']

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 150
model = Sequential()
model.add(Embedding(input_dim=wordvec_model.wv.vectors.shape[0],output_dim=wordvec_model.wv.vectors.shape[1],weights=[wordvec_model.wv.vectors], input_length=embed_size))
model.add(Bidirectional(CuDNNLSTM(64)))
#model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
#model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.summary()



Using TensorFlow backend.
d:\applications\python 3.7.1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\applications\python 3.7.1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\applications\python 3.7.1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\applications\python 3.7.1\lib\site-packages\tensorflow\python\fra

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          23045700  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               187392    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 23,233,221
Trainable params: 23,233,221
Non-trainable params: 0
_________________________________________________________________


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42,shuffle=True)
X_test

NameError: name 'X_pad' is not defined

In [ ]:
batch_size = 64
epochs = 3
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
# validate the model on test dataset to determine generalization
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

In [14]:
testdf=pd.read_csv("testData.tsv",header=0, delimiter="\t", quoting=3)
testdf.head(3)


,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."


In [17]:
testdf.to_csv('processed_testData.csv')
traindf.to_csv('processed_trainData.csv', index=False)
word2vec_traindf.to_csv('processed_fullDict.csv')


In [18]:
a = pd.read_csv('processed_testData.csv')
b = pd.read_csv('processed_labeledTrainData.csv')
c = pd.read_csv('processed_fullDict.csv')

a.head()
b.head()
c.head()

,Unnamed: 0,review,processed_review
0,0,With all this stuff going down at the moment w...,"['stuff', 'go', 'moment', 'mj', 'ive', 'start'..."
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...","['classic', 'war', 'worlds', 'timothy', 'hines..."
2,2,The film starts with a manager (Nicholas Bell)...,"['film', 'start', 'manager', 'nicholas', 'bell..."
3,3,It must be assumed that those who praised this...,"['must', 'assume', 'praise', 'film', 'greatest..."
4,4,Superbly trashy and wondrously unpretentious 8...,"['superbly', 'trashy', 'wondrously', 'unpreten..."


In [7]:
dak.restore_state()

NameError: name 'restore_state' is not defined

In [ ]:
dak.save_state()